In [129]:
from datetime import datetime, date
import math 

from IPython.display import Markdown as md
import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import curve_fit

from astropy import units as u
from astropy.coordinates import (SkyCoord, EarthLocation, AltAz, HADec)
import astropy.coordinates as coord
from astropy.time import Time

plt.rcParams['figure.figsize'] = [16,8]

%config IPCompleter.greedy=True

## Sideral Time

In [6]:
longitude=10.9836
year = datetime.utcnow().year
month = datetime.utcnow().month
day = datetime.utcnow().day
hour = datetime.utcnow().hour
minute = datetime.utcnow().minute
second = datetime.utcnow().second
UT = hour + minute / 60 + second / 3600

deltaJulianDays = julianDay(year, month, day) - julianDay(2000, 1, 1) - 0.5
deltaJulian = deltaJulianDays + hour/24 + minute/60/24 + second/3600/24

sideralTime = ((100.46 + 0.985647 * deltaJulian + longitude + 15*UT) % 360) / 15

sideralHour = math.floor(sideralTime)
sideralMinute = math.floor((sideralTime - sideralHour) * 60)
sideralSecond = ((sideralTime - sideralHour) * 60 - sideralMinute) * 60

print(f'UTC {hour:02d}:{minute:02d}:{math.floor(second):02d} => {sideralTime} => J2000 {sideralHour:02d}:{sideralMinute:02d}:{sideralSecond:.2f}')

NameError: name 'julianDay' is not defined

In [112]:
sin = math.sin
cos = math.cos

"""Calculates the Julian Day from a given gregorian date"""
def julianDay (year, month, day):
    if month in [1, 2]:
        year -= 1;
        month += 12;
        
    A = math.floor(year / 100)
    B = 2 - A + math.floor(A / 4);
    return math.floor(365.25 * (year + 4716)) + math.floor(30.6001 * (month + 1)) + day + B - 1524.5


# http://www2.arnes.si/~gljsentvid10/sidereal.htm
def localSiderealTime(year, month, day, hour, minute, second, longitude):
    deltaJulian = julianDay(year, month, day) - julianDay(2000, 1, 1) - 0.5 + hour/24 + minute/60/24 + second/3600/24
    julianCenturies = deltaJulian / 36525.
    # returns hours
    return ((280.46061837 + 360.98564736629 * deltaJulian + 0.000388 * julianCenturies**2 + longitude) % 360) / 15

def printSiderealTime(siderealTime):
    siderealHour = math.floor(siderealTime)
    siderealMinute = math.floor((siderealTime - siderealHour) * 60)
    siderealSecond = math.floor(((siderealTime - siderealHour) * 60 - siderealMinute) * 60)
    print(f'Sidereal Time (J2000) {siderealHour:02d}:{siderealMinute:02d}:{siderealSecond:02d}')
    
def rad(deg):
    return deg * math.pi / 180.

def deg(rad):
    return rad * 180. / math.pi

# http://jonvoisey.net/blog/2018/07/data-converting-alt-az-to-ra-dec-example/
def horizontalToEqatorial(azimuth, altitude, latitude, localSiderealTimeDegrees):
    az = rad(azimuth)
    alt = rad(altitude)
    lat = rad(latitude)
    
    dec = math.asin( sin(lat) * sin(alt) + cos(lat) * cos(alt) * cos(az) )
    ra = localSiderealTimeDegrees - math.acos( (sin(alt) - sin(lat) * sin(dec)) / (cos(lat) * cos(dec)) ) * 180 / math.pi
    # returns all values in degrees
    return (ra, deg(dec))

# http://www.stargazing.net/kepler/altaz.html
def equatorialToHorizontal(ra, declination, latitude, localSiderealTimeDegrees):
    hourAngle = localSiderealTimeDegrees - ra
    hourAngle = rad(hourAngle) if hourAngle >= 0  else rad(hourAngle + 360)
    dec = rad(declination)
    lat = rad(latitude)
    
    altitude = math.asin( sin(dec)*sin(lat) + cos(dec)*cos(lat)*cos(hourAngle) )

    A = math.acos( (sin(dec) - sin(altitude)*sin(lat))/(cos(altitude)*cos(lat)) ) 
    azimuth = A if sin(hourAngle) < 0  else 2 * math.pi - A
    
    # returns all values in degrees
    return(deg(azimuth), deg(altitude))

""" derived from iauHd2ae from www.iausofa.org """
def Hd2ae(ra, declination, latitude, localSiderealTimeDegrees):

    hourAngle = localSiderealTimeDegrees - ra
    ha = rad(hourAngle) if hourAngle >= 0  else rad(hourAngle + 360)
    
    dec = rad(declination)
    phi = rad(latitude)
    
    sh = sin(ha)
    ch = cos(ha)
    sd = sin(dec)
    cd = cos(dec)
    sp = sin(phi)
    cp = cos(phi)

    x = - ch*cd*sp + sd*cp;
    y = - sh*cd;
    z = ch*cd*cp + sd*sp;

    r = math.sqrt(x*x + y*y);
    
    a = math.atan2(y,x) if r != 0 else 0 # (r != 0.0) ? atan2(y,x) : 0.0;
    az = a + 2*math.pi if a < 0 else a # (a < 0.0) ? a+D2PI : a;
    el = math.atan2(z,r);
    return(deg(az), deg(el))

""" derived from iauAe2hd from www.iausofa.org """
def Ae2hd(azimuth, altitude, latitude, localSiderealTimeDegrees):
    az = rad(azimuth)
    el = rad(altitude)
    phi = rad(latitude)    
    
    sa = sin(az);
    ca = cos(az);
    se = sin(el);
    ce = cos(el);
    sp = sin(phi);
    cp = cos(phi);

    x = - ca*ce*sp + se*cp;
    y = - sa*ce;
    z = ca*ce*cp + se*sp;

    r = math.sqrt(x*x + y*y);
    ha = math.atan2(y,x) if r != 0 else 0 #(r != 0.0) ? atan2(y,x) : 0.0;
    dec = math.atan2(z,r);    
    ra = localSiderealTimeDegrees - deg(ha)
    return(ra, deg(dec))

# Sample Calculations and Testcases

## Julian Day

In [19]:
res = julianDay(2000, 1, 1) # 00:00 UTC
assert(math.isclose(res, 2451544.5, abs_tol=0.01))

In [20]:
res = julianDay(2021, 12, 23) # 00:00 UTC
assert(math.isclose(res, 2459571.5, abs_tol=0.01))

In [21]:
res = julianDay(2025, 7, 13) # 00:00 UTC
assert(math.isclose(res, 2460869.5, abs_tol=0.01))

## Sidereal Time

In [22]:
# Test case: expected result: 304.80762° from http://www.stargazing.net/kepler/altaz.html
res = localSiderealTime(1998, 8, 10, 23, 10, 0, -1.9166667) * 360 / 24 # convert to degrees
assert(math.isclose(res, 304.80762, rel_tol=0.0001))

In [23]:
# Test case: expected result: 174.77457° from http://www2.arnes.si/~gljsentvid10/sidereal.htm
res = localSiderealTime(1994, 6, 16, 18, 0, 0, 0) * 360 / 24 # convert to degrees
assert(math.isclose(res, 174.77457, rel_tol=0.0001))

In [24]:
# Test case: expected result: LST=06:39:00
res = localSiderealTime(2021, 12, 23, 8, 30, 34, -120) * 360 / 24 # convert to degrees
assert(math.isclose(res, 99.75, rel_tol=0.0001))

In [25]:
# Test case: expected result: LST=02:22:54
res = localSiderealTime(2025, 7, 13, 6, 13, 22, 11) * 360 / 24 # convert to degrees
assert(math.isclose(res, 35.7267, rel_tol=0.0001))

## Coordinate Transform

In [104]:
# Testcase #1: expected result (RA=297.92, DEC=8.93) from http://jonvoisey.net/blog/2018/07/data-converting-alt-az-to-ra-dec-example/
res = horizontalToEqatorial(azimuth=180, altitude=60.34, latitude=38.59, localSiderealTimeDegrees=297.93)
assert(math.isclose(res[0], 297.92, rel_tol=0.0001))
assert(math.isclose(res[1], 8.93, rel_tol=0.0001))

In [139]:
# Testcase #2: expected result (RA=250.425, DEC=36.4667) from http://www.stargazing.net/kepler/altaz.html
res = horizontalToEqatorial(azimuth=269.14634, altitude=49.169122, latitude=52.5, localSiderealTimeDegrees=304.80762)
assert(math.isclose(res[0], 250.425, rel_tol=0.0001))
assert(math.isclose(res[1], 36.4667, rel_tol=0.0001))

In [106]:
# Testcase #3: expected result: (AZ=269.14634, ALT=49.169122)
res = equatorialToHorizontal(ra=250.425, declination=36.467,  latitude=52.5, localSiderealTimeDegrees=304.808)
assert(math.isclose(res[0], 269.14634, rel_tol=0.0001))
assert(math.isclose(res[1], 49.169122, rel_tol=0.0001))

In [147]:
# Testcase #4: Betelgeuse (RA=05h55m10.30536s = 5.91953h = 88.7929°, DEC = +07°24′25.4304″ = 7.4071°)
res = equatorialToHorizontal(ra=88.7929, declination=7.4071,  latitude=48, localSiderealTimeDegrees=localSiderealTime(2021, 12, 23, 19, 14, 28, 11) * 360 / 24)
res
#assert(math.isclose(res[0], 269.14634, rel_tol=0.0001))
#assert(math.isclose(res[1], 49.169122, rel_tol=0.0001))

(111.07414362251865, 27.439753234662817)

In [166]:
res = horizontalToEqatorial(azimuth=110.8093, altitude=27.2852, latitude=48, localSiderealTimeDegrees=localSiderealTime(2021, 12, 23, 19, 14, 28, 11) * 360 / 24)
(res[0], res[1])
#assert(math.isclose(res[0], 250.425, rel_tol=0.0001))
#assert(math.isclose(res[1], 36.4667, rel_tol=0.0001))

(-24.743099632171237, 7.435333730727449)

In [182]:
Ae2hd(azimuth=110.8093, altitude=27.2852, latitude=48, localSiderealTimeDegrees=localSiderealTime(2021, 12, 23, 19, 14, 28, 11) * 360 / 24)

(89.0752907818754, 7.435333730727449)

In [158]:
localSiderealTime(2021, 12, 23, 19, 14, 28, 11) * 360 / 24 - 20.20605*360/24 + 360

89.07534557485207

In [184]:
(localSiderealTime(2021, 12, 23, 19, 14, 28, 11) - 20.2)%24

5.944406371656807

In [185]:
(89.0752) / 360 * 24

5.938346666666666

In [186]:
localSiderealTime(2021, 12, 23, 19, 14, 28, 11)

2.1444063716568054

# Astropy Comparison

In [153]:
observing_location = EarthLocation.from_geodetic(lon=11*u.deg, lat=48*u.deg)
observing_date = Time('2021-12-23 19:14:28')
altaz = AltAz(location=observing_location, obstime=observing_date)
betelgeuse = SkyCoord(ra=88.7929 * u.deg, dec=7.4071 * u.deg)

astropy = (betelgeuse.transform_to(altaz).az.deg, betelgeuse.transform_to(altaz).alt.deg)
simple = equatorialToHorizontal(ra=88.7929, declination=7.4071,  latitude=48, localSiderealTimeDegrees=localSiderealTime(2021, 12, 23, 19, 14, 28, 11) * 360 / 24)
tuple(map(lambda i, j: i - j, astropy, simple))

(-0.2655569125269608, -0.1871129975093453)

In [146]:
observing_location = EarthLocation.from_geodetic(lon=-1.9166667*u.deg, lat=52.5*u.deg)
observing_date = Time('1998-08-10 23:10:00')
hadec = HADec(location=observing_location, obstime=observing_date)
altaz = AltAz(location=observing_location, obstime=observing_date)
m13 = SkyCoord(ra=16.695/24*360*u.deg, dec=36.466667*u.deg)

astropy = (m13.transform_to(altaz).az.deg, m13.transform_to(altaz).alt.deg)
simple = equatorialToHorizontal(ra=250.425, declination=36.467,  latitude=52.5, localSiderealTimeDegrees=304.808)
tuple(map(lambda i, j: i - j, astropy, simple))

(0.01763446595714413, 9.608107974656832e-05)

## SOFA Library Comparison

In [121]:
iau = Ae2hd(azimuth=269.14634, altitude=49.169122, latitude=52.5, localSiderealTimeDegrees=304.80762)
simple = horizontalToEqatorial(azimuth=269.14634, altitude=49.169122, latitude=52.5, localSiderealTimeDegrees=304.80762)
tuple(map(lambda i, j: i - j, iau, simple))

(0.0, -7.105427357601002e-15)

In [122]:
iau = Hd2ae(ra=250.425, declination=36.467,  latitude=52.5, localSiderealTimeDegrees=304.808)
simple = equatorialToHorizontal(ra=250.425, declination=36.467,  latitude=52.5, localSiderealTimeDegrees=304.808)
tuple(map(lambda i, j: i - j, iau, simple))

(5.684341886080802e-14, -7.105427357601002e-15)